In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import BallTree
from math import radians

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from functools import partial


## Обработка исходных данных, путём удаления нулевых столбцов и строк, а также парсинг геопозиции пользователя


In [2]:
data = pd.read_json("../source/data.json", encoding="Windows-1251")

# Парсинг данных геолокации
data["lantitude"] = data.geoData.apply(lambda x: x["coordinates"][0])
data["longitude"] = data.geoData.apply(lambda x: x["coordinates"][1])
# Удаление ненужных столбцов
data.drop("geoData", axis=1, inplace=True)
data.drop("HelpPhoneExtension", axis=1, inplace=True)
# Удаление нулевых столбцов
data = data.loc[:, data.notna().all(axis=0)]

data.to_csv("../source/processed_data.csv", index=False)


In [ ]:
# Количество платных и бесплатных мест
data.Paid.value_counts()


In [ ]:
# Процент платных прощадок
round(len(data[data.Paid == "платно"]) / len(data) * 100, 1)


In [ ]:
# Процентовка освещений
lighting_categories = data.Lighting.unique()[1:]

all_length = len(data)
for category in lighting_categories:
    percent = int(
        round(len(data[data.Lighting == category]) / all_length, 2) * 100
    )
    print(
        f"{percent} процента[ов] составляют площадки с категорией: {category}"
    )


In [60]:
data = pd.read_csv("../source/processed_data.csv", encoding="utf-8")
data


,global_id,ObjectName,AdmArea,District,Address,Email,WebSite,HelpPhone,HasEquipmentRental,EquipmentRentalComments,...,HasWifi,HasCashMachine,HasFirstAidPost,HasMusic,Lighting,Seats,Paid,PaidComments,lantitude,longitude
0,281867778,Парк «Сад культуры и отдыха имени Н.Э. Баумана»,Центральный административный округ,Басманный район,"Старая Басманная улица, дом 15А, строение 4",sadbaumana@culture.mos.ru,sadbaumana.ru,(499) 261-58-83,нет,NaN,...,да,нет,нет,да,смешанное освещение,30,бесплатно,NaN,37.659334,55.765605
1,406671453,Парк культуры и отдыха «Фили»,Западный административный округ,район Филёвский Парк,"Большая Филёвская улица, дом 22, строение 2",info@parkfili.com,parkfili.com,(499) 145-00-00,нет,NaN,...,да,нет,нет,да,без дополнительного освещения,0,бесплатно,NaN,37.483275,55.750824
2,407118512,"Парк, озелененная городская территория «Сквер ...",Северо-Восточный административный округ,Алексеевский район,"Ракетный бульвар, дом 8",alespr@mos.ru,alekseevsky.mos.ru,(495) 620-28-20,нет,NaN,...,нет,нет,нет,нет,смешанное освещение,0,бесплатно,NaN,37.653970,55.818441
3,407200818,Дворовая территория,Западный административный округ,Можайский район,"Дорогобужская улица, дом 11",uprava-mozh@mos.ru,mozhaisky.mos.ru,(495) 447-15-60,нет,NaN,...,нет,нет,нет,нет,освещение светодиодными лампами,0,бесплатно,NaN,37.414985,55.714905
4,407262033,Дворовая территория,Юго-Западный административный округ,Ломоносовский район,"улица Вавилова, дом 78",Lomonos@uzao.mos.ru,gbu-lom.ru/,(499) 133-14-67,нет,NaN,...,нет,нет,нет,нет,без дополнительного освещения,0,бесплатно,NaN,37.544640,55.679529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,2449514710,Спортивный комплекс образовательного учреждения,Южный административный округ,район Чертаново Северное,"Кировоградская улица, дом 8Г",851@edu.mos.ru,sch851u.mskobr.ru,(495) 312-95-02,нет,NaN,...,да,нет,да,да,без дополнительного освещения,0,бесплатно,NaN,37.606504,55.627283
3213,2449514732,Спортивный комплекс образовательного учреждения,Южный административный округ,район Чертаново Южное,"Кировоградская улица, дом 42, корпус 2",1245@edu.mos.ru,sch1245u.mskobr.ru,(495) 387-22-22,нет,NaN,...,нет,нет,нет,нет,без дополнительного освещения,0,бесплатно,NaN,37.594253,55.599281
3214,2449514805,Спортивный комплекс образовательного учреждения,Северо-Западный административный округ,район Митино,"Пенягинская улица, дом 14",1544@edu.mos.ru,gym1544sz.mskobr.ru,(495) 752-61-11,нет,NaN,...,да,нет,да,нет,смешанное освещение,0,бесплатно,NaN,37.358408,55.835853
3215,2449514830,Спортивный комплекс образовательного учреждения,Юго-Западный административный округ,район Черёмушки,"Новочерёмушкинская улица, дом 48, строение 2",2115@edu.mos.ru,sch2115.mskobr.ru,(499) 128-56-33,нет,NaN,...,нет,нет,нет,нет,смешанное освещение,0,бесплатно,NaN,37.565538,55.672636


# Нахождение ближайшей площадки исходя из геопозиции пользователя.

## Алгоритм - метрическое дерево

Метод метрического дерева (англ. metric tree) — алгоритм поиска ближайших соседей, основанный на деревьях. Он используется для поиска ближайших соседей в многомерном пространстве. Основная идея метода заключается в том, что для поиска ближайших соседей достаточно найти некоторое количество точек, расстояние до которых меньше заданного порога. Для этого используется дерево, в котором каждый узел содержит некоторое количество точек, расстояние до которых меньше порога. Поиск ближайших соседей осуществляется путем обхода дерева, начиная с корня. Если расстояние до текущего узла больше порога, то ветвь дерева, в которой находится узел, отбрасывается. Если расстояние меньше порога, то ветвь дерева, в которой находится узел, рассматривается дальше. Поиск ближайших соседей заканчивается, когда все точки в дереве были рассмотрены.

Расчёт происходит с помощью гаверсинусного расстояния между двумя точками на сфере, что лучше подходит для координат, чем евклидово расстояние.


In [17]:
# Нахождение ближайших площадок исходя из геопозиции пользователя
# Алгоритм поиска соседей -  метрическое дерево.
data["lantitude_radians"] = data.lantitude.apply(lambda x: radians(x))
data["longitude_radians"] = data.longitude.apply(lambda x: radians(x))
tree = BallTree(
    data[["lantitude_radians", "longitude_radians"]], metric="haversine"
)
nearest = data.iloc[
    tree.query(
        [[radians(55.753215), radians(37.622504)]], k=15, return_distance=False
    )[0]
]


## Рекомендательная система на основе косинусного расстояния для рекомендации площадок пользователям


In [35]:
def combine_features(row, features):
    data = [f"{str(feature)}:{str(row[feature])} " for feature in features]
    return " ".join(data)


def get_index_by_name(obj_name):
    return data[data.ObjectName == obj_name].index.values[0]


def get_sorted_similarity_list(index, cosine_sim_array):
    return sorted(
        list(enumerate(cosine_sim_array[index])), key=lambda x: x[1], reverse=True
    )


def get_name_by_index(index):
    object = data[data.index == index]
    return (
        object.ObjectName.values,
        object.Address.values,
        object.Email.values,
        object.lantitude.values,
        object.longitude.values,
    )


features = list(data)[2:23]
data["combined_features"] = data.apply(
    partial(combine_features, features=features), axis=1
)

cv = CountVectorizer()
count_matrix = cv.fit_transform(data["combined_features"])
cosine_sim = cosine_similarity(count_matrix)  # бинарная мера сходства объектов

user_likes = "Парк культуры и отдыха «Фили»"

index = get_index_by_name(user_likes)

sorted_similar = get_sorted_similarity_list(index, cosine_sim)

# for index, object in enumerate(sorted_similar):
#     print(get_name_by_index(object[0]))
#     if index > 10:
#         break

df = pd.DataFrame(cosine_sim)
df

,0,1,2,3,4,5,6,7,8,9,...,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216
0,1.000000,0.723445,0.620453,0.548904,0.541994,0.544730,0.535800,0.535800,0.542678,0.544730,...,0.834332,0.809335,0.726316,0.727366,0.778663,0.819195,0.526870,0.831209,0.569094,0.569094
1,0.723445,1.000000,0.685879,0.663911,0.702998,0.694964,0.685938,0.685938,0.692346,0.694964,...,0.756768,0.773776,0.787279,0.778390,0.743269,0.741935,0.694964,0.751668,0.684739,0.684739
2,0.620453,0.685879,1.000000,0.895041,0.865226,0.871786,0.871786,0.871786,0.868502,0.871786,...,0.620637,0.715168,0.814345,0.837367,0.776899,0.617291,0.863562,0.735313,0.881865,0.881865
3,0.548904,0.663911,0.895041,1.000000,0.888038,0.893156,0.885522,0.885522,0.889792,0.893156,...,0.548638,0.654463,0.791861,0.804669,0.758090,0.554775,0.877888,0.682511,0.895760,0.895760
4,0.541994,0.702998,0.865226,0.888038,1.000000,0.965581,0.957918,0.957918,0.961944,0.965581,...,0.578298,0.666380,0.831058,0.816961,0.723900,0.566051,0.888948,0.656995,0.899225,0.899225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,0.819195,0.741935,0.617291,0.554775,0.566051,0.568607,0.559581,0.559581,0.566465,0.568607,...,0.886499,0.884315,0.776640,0.800012,0.830713,1.000000,0.613734,0.851154,0.556921,0.556921
3213,0.526870,0.694964,0.863562,0.877888,0.888948,0.893939,0.886364,0.886364,0.890572,0.893939,...,0.580763,0.686599,0.812630,0.834847,0.761500,0.613734,1.000000,0.658764,0.881284,0.881284
3214,0.831209,0.751668,0.735313,0.682511,0.656995,0.658764,0.649486,0.649486,0.656283,0.658764,...,0.844653,0.863636,0.820272,0.822425,0.853986,0.851154,0.658764,1.000000,0.685152,0.685152
3215,0.569094,0.684739,0.881865,0.895760,0.899225,0.896611,0.888948,0.888948,0.893234,0.896611,...,0.569119,0.656995,0.812991,0.807782,0.742461,0.556921,0.881284,0.685152,1.000000,1.000000


In [43]:
# Стандартный поиск по параметрам
query = "Ракетный бульвар, дом 8"

words = cv.get_feature_names_out()

querycv = cv.fit(words)

querycv = querycv.transform([query])

related = cosine_similarity(querycv, count_matrix).flatten()
sorted_related = sorted(list(enumerate(related)), key=lambda x: x[1], reverse=True)
top_related = sorted_related[:20]

for index, object in enumerate(top_related):
    
    if top_related[index][1] < 0.15:
        continue
    print(top_related[index][1])
    print(get_name_by_index(object[0]))
    if index > 10:
        break


0.1636634176769943
(array(['Парк, озелененная городская территория «Сквер по Ракетному бульвару»'],
      dtype=object), array(['Ракетный бульвар, дом 8'], dtype=object), array(['alespr@mos.ru'], dtype=object), array([37.65397001]), array([55.81844078]))
0.1636634176769943
(array(['Парк, озелененная городская территория «Сквер по Ракетному бульвару»'],
      dtype=object), array(['Ракетный бульвар, дом 8'], dtype=object), array(['alespr@mos.ru'], dtype=object), array([37.64966394]), array([55.81860457]))
